In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import json

# Load dữ liệu
docs = []
with open("../crawl_data/data/clean/myauris_allqa.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        docs.append(item["text"])

documents = [Document(page_content=doc) for doc in docs]

# Tạo embedding tiếng Việt
embedding_model = HuggingFaceEmbeddings(model_name="bkai-foundation-models/vietnamese-bi-encoder")

# Build FAISS
vectorstore = FAISS.from_documents(documents, embedding_model)

C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_14932\654496655.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="bkai-foundation-models/vietnamese-bi-encoder")
c:\Users\FPTSHOP\.conda\envs\dev\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Hàm lấy top-k đoạn văn gần nhất
def retrieve_top_k(query: str, top_k: int = 2):
    retriever = vectorstore.as_retriever(search_type="similarity", k=top_k)
    results = retriever.get_relevant_documents(query)
    return results

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
from together import Together

client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

def generate_answer(query, docs, model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free"):
    context = "\n".join([doc.page_content for doc in docs])
    prompt = f"""Bạn là trợ lý tư vấn nha khoa của My Auris. Dưới đây là các thông tin liên quan:
{context}

Câu hỏi: {query}
Hãy trả lời rõ ràng, đúng nội dung, bằng tiếng Việt:"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=None
    )

    return response.choices[0].message.content

In [4]:
query = "Giá niềng răng là bao nhiêu?"
top_k_docs = retrieve_top_k(query, top_k=3)
answer = generate_answer(query, top_k_docs)

print("Câu trả lời:", answer)

C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_14932\2801517889.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


Câu trả lời: <think>
Được rồi, tôi đang xem xét câu hỏi của khách hàng về giá niềng răng tại Nha khoa My Auris. Họ muốn biết chi phí cụ thể cho dịch vụ này. Từ thông tin mà tôi có, tôi nhớ rằng My Auris cung cấp một số phương pháp niềng răng khác nhau với các mức giá đã được quy định rõ ràng.

Trước tiên, tôi cần xác định xem khách hàng có đang tìm kiếm thông tin về các loại niềng răng khác nhau không. Tại My Auris, có ba loại niềng răng chính: kim loại thường, kim loại tự khóa và mắc cài sứ tự khóa. Mỗi loại có chi phí riêng.

Niềng răng kim loại thường có giá 35.000.000 VNĐ, đây là lựa chọn tiết kiệm nhất. Niềng răng kim loại tự khóa có giá cao hơn một chút, 45.000.000 VNĐ, vì nó sử dụng công nghệ tự khóa hiện đại, giúp quá trình điều trị nhanh hơn và thoải mái hơn. Cuối cùng, niềng răng mắc cài sứ tự khóa là lựa chọn cao cấp nhất, với chi phí 60.000.000 VNĐ, phù hợp với những người muốn tính thẩm mỹ cao hơn vì mắc cài bằng sứ có màu sắc tương tự như răng tự nhiên.

Tôi cũng cần nhấn